In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [1]:
# connect gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Paths
NL_Dataset_words_dir = "/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/"

df_path = os.path.join(NL_Dataset_words_dir, 'Huth_fMRI_25_ordered_stories_SCOPEmerged_Spacy_tagged_trimmed_df.csv')
dataframe = pd.read_csv(df_path)
display(dataframe)


,onset,offset,word_cap,story,onset_TR,offset_TR,word,Conc_Brys,Freq_SUBTLEXUS,PoS_tag_nltk,ptb_tag,upos,pos_source,is_content
0,10.069615,10.129478,TO,adollshouse,5,5,to,1.55,6.063172,TO,TO,PART,orig,False
1,10.129478,10.219274,MEET,adollshouse,5,5,meet,3.00,4.254476,VB,VB,VERB,orig,True
2,10.219274,10.299093,MY,adollshouse,5,5,my,2.42,5.537693,PRP$,PRP$,PRON,orig,True
3,10.299093,10.847846,PARENTS,adollshouse,5,5,parents,4.56,3.856003,NNS,NNS,NOUN,orig,True
4,11.656009,11.725850,SHE,adollshouse,5,5,she,3.36,5.279617,PRP,PRP,PRON,orig,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51948,590.470522,590.630159,I,wheretheressmoke,295,295,i,3.93,6.309317,PRP,PRP,PRON,orig,True
51949,590.630159,590.999320,STILL,wheretheressmoke,295,295,still,3.46,4.604507,RB,RB,ADV,orig,True
51950,590.999320,591.188889,MISS,wheretheressmoke,295,295,miss,2.54,4.377507,JJ,VBP,VERB,orig,True
51951,591.188889,591.298639,THE,wheretheressmoke,295,295,the,1.43,6.176644,DT,DT,DET,orig,False


['adollshouse' 'adventuresinsayingyes' 'alternateithicatom' 'avatar'
 'buck' 'exorcism' 'eyespy' 'hangtime' 'haveyoumethimyet' 'howtodraw'
 'inamoment' 'itsabox' 'legacy' 'myfirstdaywiththeyankees' 'naked'
 'odetostepfather' 'sloth' 'souls' 'stagefright' 'swimmingwithastronauts'
 'thatthingonmyarm' 'theclosetthatateeverything' 'tildeath'
 'undertheinfluence' 'wheretheressmoke']


In [13]:
print(f"Unique stories in the dataframe:\n{dataframe['story'].unique()}\n")
print(f"Story count: {len(dataframe['story'].unique())}\n")

Unique stories in the dataframe:
['adollshouse' 'adventuresinsayingyes' 'alternateithicatom' 'avatar'
 'buck' 'exorcism' 'eyespy' 'hangtime' 'haveyoumethimyet' 'howtodraw'
 'inamoment' 'itsabox' 'legacy' 'myfirstdaywiththeyankees' 'naked'
 'odetostepfather' 'sloth' 'souls' 'stagefright' 'swimmingwithastronauts'
 'thatthingonmyarm' 'theclosetthatateeverything' 'tildeath'
 'undertheinfluence' 'wheretheressmoke']

Story count: 25



In [14]:
dataframe.columns

Index(['onset', 'offset', 'word_cap', 'story', 'onset_TR', 'offset_TR', 'word',
       'Conc_Brys', 'Freq_SUBTLEXUS', 'PoS_tag_nltk', 'ptb_tag', 'upos',
       'pos_source', 'is_content'],
      dtype='object')

# Getting word emebeddings for each word using fasttext

In [15]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498215 sha256=72c032869683035b59e5678a19575ea7a545ae0b157e63b61909bcfa0de3d7ca
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


In [17]:
# # I need to fasttext bin in the drive
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P /content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext
# !gunzip /content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/cc.en.300.bin.gz

# Already downloaded it once. Don't need to run this again.

--2025-11-23 00:39:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.51, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  32.8MB/s    in 2m 3s   

2025-11-23 00:41:26 (35.0 MB/s) - ‘/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [18]:
import fasttext

In [19]:
ft_model_path = "/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/cc.en.300.bin"
model = fasttext.load_model(ft_model_path)


In [20]:
def get_ft_embedding(word: str):
    if not isinstance(word, str):
        return np.zeros(300, dtype=np.float32)
    return model.get_word_vector(word)

dataframe["fasttext_embedding"] = dataframe["word"].apply(get_ft_embedding)

In [21]:
display(dataframe)
print(dataframe.columns)

,onset,offset,word_cap,story,onset_TR,offset_TR,word,Conc_Brys,Freq_SUBTLEXUS,PoS_tag_nltk,ptb_tag,upos,pos_source,is_content,fasttext_embedding
0,10.069615,10.129478,TO,adollshouse,5,5,to,1.55,6.063172,TO,TO,PART,orig,False,"[0.0046811374, 0.02812425, -0.029631453, -0.01..."
1,10.129478,10.219274,MEET,adollshouse,5,5,meet,3.00,4.254476,VB,VB,VERB,orig,True,"[0.09194227, 0.09071794, -0.16668129, 0.128149..."
2,10.219274,10.299093,MY,adollshouse,5,5,my,2.42,5.537693,PRP$,PRP$,PRON,orig,True,"[0.1964688, -0.031372625, -0.24117081, 0.04461..."
3,10.299093,10.847846,PARENTS,adollshouse,5,5,parents,4.56,3.856003,NNS,NNS,NOUN,orig,True,"[0.01267105, 0.007303595, -0.033654325, 0.0359..."
4,11.656009,11.725850,SHE,adollshouse,5,5,she,3.36,5.279617,PRP,PRP,PRON,orig,True,"[-0.012961192, 0.047705665, 0.0065072835, 0.15..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51948,590.470522,590.630159,I,wheretheressmoke,295,295,i,3.93,6.309317,PRP,PRP,PRON,orig,True,"[0.0030313858, -0.1436925, -0.16985153, 0.0111..."
51949,590.630159,590.999320,STILL,wheretheressmoke,295,295,still,3.46,4.604507,RB,RB,ADV,orig,True,"[-0.030767146, -0.05990688, -0.031013027, 0.14..."
51950,590.999320,591.188889,MISS,wheretheressmoke,295,295,miss,2.54,4.377507,JJ,VBP,VERB,orig,True,"[0.039345313, -0.030646069, -0.03299479, 0.113..."
51951,591.188889,591.298639,THE,wheretheressmoke,295,295,the,1.43,6.176644,DT,DT,DET,orig,False,"[-0.051744193, 0.073963955, -0.01305688, 0.044..."


Index(['onset', 'offset', 'word_cap', 'story', 'onset_TR', 'offset_TR', 'word',
       'Conc_Brys', 'Freq_SUBTLEXUS', 'PoS_tag_nltk', 'ptb_tag', 'upos',
       'pos_source', 'is_content', 'fasttext_embedding'],
      dtype='object')


In [22]:
dataframe.to_csv('/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/Huth_fMRI_25_ordered_stories_SCOPEmerged_Spacy_tagged_trimmed_fasttext_df.csv')

In [27]:
# just load the above dataframe.

dataframe = pd.read_csv('/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/Huth_fMRI_25_ordered_stories_SCOPEmerged_Spacy_tagged_trimmed_fasttext_df.csv').drop(columns=['Unnamed: 0'])
display(dataframe)

,onset,offset,word_cap,story,onset_TR,offset_TR,word,Conc_Brys,Freq_SUBTLEXUS,PoS_tag_nltk,ptb_tag,upos,pos_source,is_content,fasttext_embedding
0,10.069615,10.129478,TO,adollshouse,5,5,to,1.55,6.063172,TO,TO,PART,orig,False,[ 4.68113739e-03 2.81242505e-02 -2.96314526e-...
1,10.129478,10.219274,MEET,adollshouse,5,5,meet,3.00,4.254476,VB,VB,VERB,orig,True,[ 0.09194227 0.09071794 -0.16668129 0.128149...
2,10.219274,10.299093,MY,adollshouse,5,5,my,2.42,5.537693,PRP$,PRP$,PRON,orig,True,[ 1.96468800e-01 -3.13726254e-02 -2.41170809e-...
3,10.299093,10.847846,PARENTS,adollshouse,5,5,parents,4.56,3.856003,NNS,NNS,NOUN,orig,True,[ 0.01267105 0.0073036 -0.03365432 0.035943...
4,11.656009,11.725850,SHE,adollshouse,5,5,she,3.36,5.279617,PRP,PRP,PRON,orig,True,[-1.29611921e-02 4.77056652e-02 6.50728354e-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51948,590.470522,590.630159,I,wheretheressmoke,295,295,i,3.93,6.309317,PRP,PRP,PRON,orig,True,[ 0.00303139 -0.1436925 -0.16985153 0.011115...
51949,590.630159,590.999320,STILL,wheretheressmoke,295,295,still,3.46,4.604507,RB,RB,ADV,orig,True,[-3.07671465e-02 -5.99068813e-02 -3.10130268e-...
51950,590.999320,591.188889,MISS,wheretheressmoke,295,295,miss,2.54,4.377507,JJ,VBP,VERB,orig,True,[ 3.93453129e-02 -3.06460690e-02 -3.29947881e-...
51951,591.188889,591.298639,THE,wheretheressmoke,295,295,the,1.43,6.176644,DT,DT,DET,orig,False,[-5.1744193e-02 7.3963955e-02 -1.3056880e-02 ...


In [29]:
def ensure_numpy(x):
    if isinstance(x, np.ndarray):
        return x.astype(np.float32)
    if isinstance(x, list):
        return np.array(x, dtype=np.float32)
    if isinstance(x, str):  # stored as "[0.1 0.2 ...]"
        # safest way
        return np.fromstring(x.strip("[]"), sep=' ', dtype=np.float32)
    raise ValueError(f"Unknown embedding type: {type(x)}")

dataframe["fasttext_embedding"] = dataframe["fasttext_embedding"].apply(ensure_numpy)



1.   To do TR level anlaysis we will pool all the words' emebddings in a TR.
2.   Also, if there are missing TRs; then we will populate that TR with zero array of the same length.
3.   This ensures uniforminty that all fMRI data will have corresponding stimuli informations.



In [30]:
# TR collapse. We will pool all words belonging to a TR.

def build_TR_level_embeddings_for_story(df_story):
    """
    df_story: dataframe for a single story.
              Must contain columns: onset_TR, fasttext_embedding (np.array shape 300)
    Returns: dataframe with TR (int) and embedding (np.array shape 300)
             Full TR range is preserved (zero-filled where missing).
    """

    # --- A) Get min/max TR for full coverage ---
    tr_min = df_story["onset_TR"].min()
    tr_max = df_story["onset_TR"].max()

    # --- B) Group by TR and average embeddings ---
    tr_groups = (
        df_story
        .groupby("onset_TR")["fasttext_embedding"]
        .apply(lambda x: np.mean(np.vstack(x), axis=0))
    )  # This becomes a Series: index = TR, value = np.array(300)

    # --- C) Prepare output with full TR range ---
    all_TRs = np.arange(tr_min, tr_max + 1)  # inclusive

    zero_vec = np.zeros(300, dtype=np.float32)

    averaged_embeddings = []
    for tr in all_TRs:
        if tr in tr_groups.index:
            averaged_embeddings.append(tr_groups.loc[tr])
        else:
            averaged_embeddings.append(zero_vec)  # missing TR → zero fill

    # --- D) Return dataframe ---
    out_df = pd.DataFrame({
        "TR": all_TRs,
        "embedding": averaged_embeddings
    })

    return out_df

In [31]:
# Get list of unique stories
stories = dataframe["story"].unique()

story_to_TR_embeddings = {}  # dict: {story → TR-level dataframe}

for story in stories:
    df_story = dataframe[dataframe["story"] == story]
    tr_df = build_TR_level_embeddings_for_story(df_story)
    story_to_TR_embeddings[story] = tr_df

    print(f"{story}: TRs = {tr_df.shape[0]}, range = {tr_df['TR'].min()}–{tr_df['TR'].max()}")

adollshouse: TRs = 241, range = 5–245
adventuresinsayingyes: TRs = 391, range = 5–395
alternateithicatom: TRs = 343, range = 5–347
avatar: TRs = 366, range = 5–370
buck: TRs = 332, range = 5–336
exorcism: TRs = 466, range = 5–470
eyespy: TRs = 379, range = 5–383
hangtime: TRs = 324, range = 5–328
haveyoumethimyet: TRs = 494, range = 7–500
howtodraw: TRs = 353, range = 5–357
inamoment: TRs = 205, range = 5–209
itsabox: TRs = 355, range = 5–359
legacy: TRs = 400, range = 5–404
myfirstdaywiththeyankees: TRs = 358, range = 5–362
naked: TRs = 422, range = 5–426
odetostepfather: TRs = 404, range = 5–408
sloth: TRs = 437, range = 5–441
souls: TRs = 355, range = 5–359
stagefright: TRs = 293, range = 5–297
swimmingwithastronauts: TRs = 385, range = 5–389
thatthingonmyarm: TRs = 434, range = 5–438
theclosetthatateeverything: TRs = 314, range = 5–318
tildeath: TRs = 323, range = 5–327
undertheinfluence: TRs = 303, range = 5–307
wheretheressmoke: TRs = 291, range = 5–295
